In [1]:
# install required dependencies
%pip install kagglehub
%pip install pandas
%pip install nltk
%pip install sklearn
%pip install tensorflow
%pip install matplotlib
%pip install tf-keras

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
# import all the required dependencies
import kagglehub
import random
import numpy as np
import pandas as pd
import regex as re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from multiprocessing import Pool, cpu_count

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# Download dataset
path = kagglehub.dataset_download("tboyle10/medicaltranscriptions")
print("Path to dataset files:", path)

dataset = pd.read_csv(path + "/mtsamples.csv")
print("Head: ", dataset.head)

100%|██████████| 4.85M/4.85M [00:00<00:00, 32.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tboyle10/medicaltranscriptions/versions/1
Head:  <bound method NDFrame.head of       Unnamed: 0                                        description  \
0              0   A 23-year-old white female presents with comp...   
1              1           Consult for laparoscopic gastric bypass.   
2              2           Consult for laparoscopic gastric bypass.   
3              3                             2-D M-Mode. Doppler.     
4              4                                 2-D Echocardiogram   
...          ...                                                ...   
4994        4994   Patient having severe sinusitis about two to ...   
4995        4995   This is a 14-month-old baby boy Caucasian who...   
4996        4996   A female for a complete physical and follow u...   
4997        4997   Mother states he has been wheezing and coughing.   
4998        4998   Acute allergic reaction, etiology uncertain, ...   

             

In [4]:
# Drop rows with missing values in specified columns
dataset.dropna(subset=['transcription', 'medical_specialty'], inplace=True)

# Keep only relevant columns
dataset = dataset[['transcription', 'medical_specialty']]

# # Filter medical specialties with at least 30 occurrences
specialty_counts = dataset['medical_specialty'].value_counts()
# valid_specialties = specialty_counts[specialty_counts >= 50].index
# dataset = dataset[dataset['medical_specialty'].isin(valid_specialties)]

# Strip spaces in 'medical_specialty' column
dataset['medical_specialty'] = dataset['medical_specialty'].str.strip()

# Remove specific categories
excluded_specialties = [
    'Surgery',
    'SOAP / Chart / Progress Notes',
    'Office Notes',
    'Consult - History and Phy.',
    'Emergency Room Reports',
    'Discharge Summary',
    'Pain Management',
    'General Medicine',
    'Radiology',
]

dataset = dataset[~dataset['medical_specialty'].isin(excluded_specialties)]

# Define category mapping to merge similar categories
category_mapping = {
    'Neurosurgery': 'Neurology',
    'Nephrology': 'Urology',
}

# Apply category mapping
dataset['medical_specialty'] = dataset['medical_specialty'].replace(category_mapping)

# Display counts for each category
for i, (category_name, category) in enumerate(dataset.groupby("medical_specialty")):
    print(f"Category {i}: {category_name}: {len(category)}")

Category 0: Allergy / Immunology: 7
Category 1: Autopsy: 8
Category 2: Bariatrics: 18
Category 3: Cardiovascular / Pulmonary: 371
Category 4: Chiropractic: 14
Category 5: Cosmetic / Plastic Surgery: 27
Category 6: Dentistry: 27
Category 7: Dermatology: 29
Category 8: Diets and Nutritions: 10
Category 9: ENT - Otolaryngology: 96
Category 10: Endocrinology: 19
Category 11: Gastroenterology: 224
Category 12: Hematology - Oncology: 90
Category 13: Hospice - Palliative Care: 6
Category 14: IME-QME-Work Comp etc.: 16
Category 15: Lab Medicine - Pathology: 8
Category 16: Letters: 23
Category 17: Neurology: 317
Category 18: Obstetrics / Gynecology: 155
Category 19: Ophthalmology: 83
Category 20: Orthopedic: 355
Category 21: Pediatrics - Neonatal: 70
Category 22: Physical Medicine - Rehab: 21
Category 23: Podiatry: 47
Category 24: Psychiatry / Psychology: 53
Category 25: Rheumatology: 10
Category 26: Sleep Medicine: 20
Category 27: Speech - Language: 9
Category 28: Urology: 237


In [5]:
# Helper: Get synonyms from WordNet
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ')
            if synonym.lower() != word.lower():
                synonyms.add(synonym)
    return list(synonyms)

# Synonym Replacement with random n
def synonym_replacement(text, n=None):
    words = word_tokenize(text)
    new_words = words.copy()
    eligible_words = list(set([word for word in words if word.isalpha()]))
    random.shuffle(eligible_words)

    if n is None:
        n = random.randint(1, min(3, len(eligible_words)))

    num_replaced = 0
    for word in eligible_words:
        synonyms = get_synonyms(word)
        if synonyms:
            synonym = random.choice(synonyms)
            new_words = [synonym if w == word else w for w in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return ' '.join(new_words)

# Word Dropout
def word_dropout(text, dropout_prob=0.1):
    words = word_tokenize(text)
    new_words = [word for word in words if random.random() > dropout_prob]
    return ' '.join(new_words) if new_words else text

# Random Swap
def random_swap(text, n=1):
    words = word_tokenize(text)
    if len(words) < 2:
        return text
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

# Apply 1–2 random augmentations
def augment_text_randomly(text):
    aug_functions = [
        synonym_replacement,
        word_dropout,
        random_swap
    ]
    num_augs = random.randint(1, 2)
    selected_augs = random.sample(aug_functions, num_augs)
    for aug in selected_augs:
        text = aug(text)
    return text

# Parallel augmentation per class
def augment_class(label_samples_tuple):
    label, samples_needed, class_samples = label_samples_tuple
    samples = class_samples.sample(n=samples_needed, replace=True)
    texts = samples['transcription'].tolist()

    with Pool(cpu_count()) as p:
        augmented_texts = p.map(augment_text_randomly, texts)

    return pd.DataFrame({
        'transcription': augmented_texts,
        'medical_specialty': label
    })

specialty_counts = dataset['medical_specialty'].value_counts()
max_count = specialty_counts.max()

# Prepare augmentation tasks
tasks = []
for label, count in specialty_counts.items():
    if count < max_count:
        samples_needed = max_count - count
        class_samples = dataset[dataset['medical_specialty'] == label]
        tasks.append((label, samples_needed, class_samples))

# Run augmentations
augmented_dfs = [augment_class(task) for task in tasks]

# Combine datasets
augmented_df = pd.concat(augmented_dfs, ignore_index=True)
dataset = pd.concat([dataset, augmented_df], ignore_index=True)

In [6]:
from sklearn.model_selection import train_test_split

lemmatizer = None
stop_words = None

def init_worker():
    global lemmatizer, stop_words
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.strip().lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

def apply_multiprocessing(series, func, workers=None):
    with Pool(processes=workers or cpu_count(), initializer=init_worker) as pool:
        results = pool.map(func, series)
    return results

# Use multiprocessing to speed it up
dataset['processed_transcription'] = apply_multiprocessing(dataset['transcription'], clean_text)

X_train, X_test, y_train, y_test = train_test_split(
    dataset['processed_transcription'], dataset['medical_specialty'], test_size=0.2, random_state=42, stratify=dataset['medical_specialty']
)

# Random Forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

rf_pipeline = Pipeline([('tfidf', TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 5), max_df=0.7, max_features=2000)),
                        ('clf', RandomForestClassifier(n_estimators=150, random_state=30))])

rf_pipeline.fit(X_train, y_train)

rf_y_pred = rf_pipeline.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_y_pred)
print(f'Random Forest Accuracy: {rf_accuracy:.4f}')
print(classification_report(y_test, rf_y_pred))

Random Forest Accuracy: 0.8954
                            precision    recall  f1-score   support

      Allergy / Immunology       0.99      1.00      0.99        74
                   Autopsy       1.00      0.99      0.99        74
                Bariatrics       0.95      0.97      0.96        74
Cardiovascular / Pulmonary       0.82      0.85      0.83        74
              Chiropractic       0.78      0.91      0.84        74
Cosmetic / Plastic Surgery       0.86      0.88      0.87        74
                 Dentistry       1.00      1.00      1.00        75
               Dermatology       0.90      0.92      0.91        75
      Diets and Nutritions       1.00      1.00      1.00        74
      ENT - Otolaryngology       0.86      0.91      0.88        74
             Endocrinology       0.93      0.99      0.95        75
          Gastroenterology       0.87      0.73      0.79        74
     Hematology - Oncology       0.81      0.78      0.79        74
 Hospice - Palli

# Bi-LSTM

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import numpy as np

max_words = 50000
max_len = 2000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)

y_train_cat = tf.keras.utils.to_categorical(y_train_enc)
y_test_cat = tf.keras.utils.to_categorical(y_test_enc)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, GlobalMaxPooling1D, LayerNormalization

model = Sequential([Embedding(input_dim=max_words, output_dim=128, input_length=max_len), Bidirectional(LSTM(64, return_sequences=True, dropout=0.3)),
                    LayerNormalization(), GlobalMaxPooling1D(), Dropout(0.4),
                    Dense(64, activation='relu'), Dropout(0.4), Dense(y_train_cat.shape[1], activation='softmax')])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
history = model.fit(X_train_pad, y_train_cat, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/20
208/216 ━━━━━━━━━━━━━━━━━━━━ 30s 4s/step - accuracy: 0.0551 - loss: 3.4543

In [ ]:
loss, accuracy = model.evaluate(X_test_pad, y_test_cat, verbose=0)
print("BiLSTM Accuracy (test): ", accuracy)

y_pred_probs = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

y_true_classes = np.argmax(y_test_cat, axis=1)

# get original string names
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_true_labels = label_encoder.inverse_transform(y_true_classes)

from sklearn.metrics import classification_report
print(classification_report(y_true_labels, y_pred_labels))

# GRU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Bidirectional, Dense, Dropout
from tensorflow.keras.layers import LayerNormalization, GlobalMaxPooling1D

model = Sequential([Embedding(max_words, 128, input_length=max_len), Bidirectional(GRU(64, return_sequences=True)),
    LayerNormalization(), GlobalMaxPooling1D(), Dropout(0.4),
    Dense(64, activation='relu'), Dropout(0.4), Dense(y_train_cat.shape[1], activation='softmax')])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train_pad, y_train_cat, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
loss, accuracy = model.evaluate(X_test_pad, y_test_cat, verbose=0)
print("GRU Accuracy (test): ", accuracy)

y_pred_probs = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test_cat, axis=1)

# Decode to string labels
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_true_labels = label_encoder.inverse_transform(y_true_classes)

from sklearn.metrics import classification_report
print(classification_report(y_true_labels, y_pred_labels))